In [ ]:
import pandas as pd
import re

In [ ]:
df =  pd.read_csv('atriographics_030225.csv', sep = ";", encoding="utf-8")
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
lista = df["Descripción"].str.split().str[0]

In [ ]:
sorted(lista.unique())

In [ ]:
len(lista.unique())

In [ ]:
mapeo_eventos = {
    '(Sesiones': lambda x: "CONEXION" if re.search(r'\bconectada\b', x.lower()) else "DESCONEXION",
    'Aceptar': "ACEPTAR_ACCION",       
    'Añadida': "AÑADIR_PIEZA",
    'Borrado': "BORRAR_ITEM",          
    'Cambio': "CAMBIOS",              
    'Cargado': "CARGAR_DISENO",
    'Creado': "CREAR_DISENO",
    'Duración': "DURACION",             
    'Error': "ERROR_SISTEMA",          
    'Generar': lambda x: "GEN_PRESUPUESTO" if re.search(r'\bpresupuesto\b', x.lower()) else "GEN_PEDIDO",
    'Intento': "INTENTO_ACCION",       
    'Limpiar': "LIMPIAR_DATOS",        
    'Login': "LOGIN",
    'Logout': "LOGOUT",
    'Nuevo': "NUEVO_ITEM",              
    'Solicitud': "REQUEST",            
    'Visualizar': lambda x: "VER_PRESUPUESTO" if re.search(r'\bpresupuesto\b', x.lower()) else "VER_PEDIDO",
}

In [ ]:
def categorizar_evento(descripcion):
    descripcion = str(descripcion)
    primera_palabra = descripcion.split()[0]
    
    if primera_palabra in mapeo_eventos:
        if callable(mapeo_eventos[primera_palabra]):
            return mapeo_eventos[primera_palabra](descripcion)
        else:
            return mapeo_eventos[primera_palabra]
    else:
        return "OTRO"

In [ ]:
df["Evento"] = df["Descripción"].apply(categorizar_evento)

In [ ]:
df[df['Evento'] == 'DURACION'][['Descripción', 'Evento']].head(10)

In [ ]:
def extraer_info_general(row):
    evento = row['Evento']
    desc = row['Descripción']
    if pd.isna(desc):
        return {}

    if evento == 'AÑADIR_PIEZA':
        return {
            'pieza': buscar(r'pieza ([\w-]+)', desc),
            'piezaid': buscar(r'id (\d+)', desc),
            'catalogo': buscar(r'catálogo (.*?) en el diseño', desc),
            'diseñoid': buscar(r'diseño con id: (-?\d+)', desc)
        }

    elif evento == 'LOGIN' or evento == 'LOGOUT' or evento == 'REQUEST' or evento == 'ACEPTAR_ACCION' or evento == 'CAMBIOS':
        return {
            'userid': buscar(r'userID: (\w+)', desc)
        }
    elif evento == 'GEN_PEDIDO':
        return {
            'diseñoid': buscar(r'diseño con id: (-?\d+)', desc),
            'fabricante': buscar(r'fabricante (.*)', desc)
        }

    elif evento == 'CREAR_DISENO' or evento == 'CARGAR_DISENO' or evento == 'GEN_PRESUPUESTO' or evento == 'VER_PRESUPUESTO' or evento == 'ERROR_SISTEMA' or evento == 'BORRAR_ITEM' or evento == 'VER_PEDIDO':
        return {
            'diseñoid': buscar(r'diseño con id: (-?\d+)', desc)
        }

    elif evento == 'INTENTO_ACCION':
        return {
            'email': buscar(r'email[:\s]+(\S+)', desc)
        }

    elif evento == 'DURACION':
        return {
            'minutos': buscar(r'Duración de la sesión:\s*([\d.]+)', desc)
        }

    elif evento == 'NUEVO_ITEM':
        return {
            'logo': buscar(r'logo[:\s]+(\w+)', desc)
        }

    elif evento == 'CONEXION' or evento == 'DESCONEXION':
        return {
            'nsesiones': buscar(r'activas (\d+)', desc)
        }
    return {}
def buscar(patron, texto):
    match = re.search(patron, texto)
    return match.group(1) if match else None

In [ ]:
df_extra = df.apply(extraer_info_general, axis=1, result_type='expand')

In [ ]:
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y %H:%M', errors='coerce')

In [ ]:
df = pd.concat([df, df_extra], axis=1)

In [ ]:
tipo_numerico = ['minutos']
df[tipo_numerico] = df[tipo_numerico].apply(pd.to_numeric, errors='coerce')

In [ ]:
tipo_int = ['nsesiones', 'piezaid', 'diseñoid', 'userid']
df[tipo_int] = df[tipo_int].apply(lambda x: x.astype('Int64'))

In [ ]:
df.info()

In [ ]:
df.to_parquet("logs.parquet",index=False)

In [ ]:
df.to_csv("logs.csv",index=False)